In [1]:
#6
#athena db config - lifearchi
#AAI-540 Group 3 FP

In [2]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [4]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_8292/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [5]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_8292/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [6]:
s3_data_path = 's3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv'
s3_table_path = 's3://sagemaker-us-east-1-342942554664/table3/'

In [7]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv to s3://sagemaker-us-east-1-342942554664/table3/lifearchitectmodels.csv


In [8]:
#table3 - lifearch
table_name_csv = 'lifearchitect'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('lifearchitectmodels.csv')
dataexplore.head()

,model,lab,parameters_b,tokens_trained_b,ratio_tokens_params,alscore,mmlu,mmlu_pro,gpqa,hle,announced,public,arch,tags
0,Model,Lab,Parameters \n(B),Tokens \ntrained (B),Ratio Tokens:Params\n(Chinchilla scaling≥20:1),"ALScore \n""ALScore"" is a quick and dirty ratin...",MMLU,MMLU\n-Pro,GPQA,HLE,Announced\n▼,Public?,Arch,Tags
1,AuroraGPT (ScienceGPT),Argonne National Laboratory,2000,30000,15:1,25.8,NaN,NaN,NaN,NaN,TBA,🔴,NaN,NaN
2,Avocado,Meta AI,2000,30000,NaN,NaN,NaN,NaN,NaN,NaN,TBA,NaN,NaN,NaN
3,DeepSeek-R2,DeepSeek-AI,1200,15640,14:1,14.4,NaN,NaN,NaN,NaN,TBA,🟢,MoE,"Reasoning, SOTA"
4,GPT-6,OpenAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TBA,NaN,NaN,SOTA


In [9]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   model                754 non-null    object
 1   lab                  753 non-null    object
 2   parameters_b         735 non-null    object
 3   tokens_trained_b     661 non-null    object
 4   ratio_tokens_params  658 non-null    object
 5   alscore              658 non-null    object
 6   mmlu                 249 non-null    object
 7   mmlu_pro             145 non-null    object
 8   gpqa                 192 non-null    object
 9   hle                  53 non-null     object
 10  announced            753 non-null    object
 11  public               750 non-null    object
 12  arch                 749 non-null    object
 13  tags                 190 non-null    object
dtypes: object(14)
memory usage: 82.6+ KB


In [10]:
!aws s3 cp s3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv \
           s3://sagemaker-us-east-1-342942554664/table3/

copy: s3://sagemaker-us-east-1-342942554664/lifearchitectmodels.csv to s3://sagemaker-us-east-1-342942554664/table3/lifearchitectmodels.csv


In [11]:
!aws s3 ls s3://sagemaker-us-east-1-342942554664/table3/

2026-02-12 09:51:51      51984 lifearchitectmodels.csv
2026-02-12 09:22:20      18853 llmachievements.csv


In [12]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         model STRING,
         lab STRING,
         parameters_b STRING,
         tokens_trained_b STRING,
         ratio_tokens_params STRING,
         alscore STRING,
         mmlu STRING,
         mmlu_pro STRING,
         gqpa STRING,
         hle STRING,
         announced STRING,
         public STRING,
         arch STRING,
         tags STRING
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [13]:
pd.read_sql(statement, conn)

/tmp/ipykernel_8292/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [14]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_8292/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,lifearchitect
2,lifearchitectmodels
3,llmachievements


In [15]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.lifearchitect LIMIT 10


/tmp/ipykernel_8292/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,model,lab,parameters_b,tokens_trained_b,ratio_tokens_params,alscore,mmlu,mmlu_pro,gqpa,hle,announced,public,arch,tags
0,Music,97% of people can’t tell the difference betwee...,,,Yes,Multiple,Nov/2025,"""""""all participants were asked to listen to th...",None,None,None,None,None,None
1,Transcription,Transcribing handwritten historical documents.,99.44,96.0,Yes,Gemini 3,Nov/2025,"""""""The new Gemini model’s performance on HTR m...",the new Gemini model was only getting about 1...,not counting punctuation marks and capital le...,None,None,None,None
2,Finance,Large Language Models pass CFA Level III.,79.1,50.0,Yes,o4-mini,Jul/2025,"""""""leading models demonstrate strong capabilities",with composite scores such as 79.1% (o4-mini)...,None,None,None,None,None
3,CBRN,LLMs can can accurately guide users through th...,,,Yes,GPT-4o,Jun/2025,"""""""we find that advanced AI models Llama 3.1 405B",ChatGPT-4o,and Claude 3.5 Sonnet can accurately guide us...,None,None,None,None
4,Health reviews,LLMs outperform humans in synthesizing results...,96.7,81.7,Yes,o3-mini-high,Jun/2025,"""""""We developed otto-SR",an end-to-end agentic workflow using large la...,97.9% specificity; human: 81.7% sensitivity,98.1% specificity) and data extraction (otto-...,we reproduced and updated an entire issue of ...,representing approximately 12 work-years of t...,otto-SR incorrectly excluded a median of 0 st...
